# Activity 2.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [ ]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [ ]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [ ]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [ ]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [ ]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()



11490434/11490434 [==============================] - 0s 0us/step


Create two datasets
* one with digits below 5
* one with 5 and above

In [ ]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [ ]:


feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [ ]:


classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [ ]:

model = Sequential(feature_layers + classification_layers)

Check the model summary

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 51s 212ms/step - loss: 1.6017 - accuracy: 0.2265 - val_loss: 1.5865 - val_accuracy: 0.3462
Epoch 2/5
230/230 [==============================] - 45s 196ms/step - loss: 1.5842 - accuracy: 0.2816 - val_loss: 1.5664 - val_accuracy: 0.4937
Epoch 3/5
230/230 [==============================] - 48s 206ms/step - loss: 1.5657 - accuracy: 0.3382 - val_loss: 1.5441 - val_accuracy: 0.5968
Epoch 4/5
230/230 [==============================] - 46s 200ms/step - loss: 1.5439 - accuracy: 0.3982 - val_loss: 1.5187 - val_accuracy: 0.6700
Epoch 5/5
230/230 [==============================] - 44s 191ms/step - loss: 1.5185 - accuracy: 0.4611 - val_loss: 1.4894 - val_accuracy: 0.7182
Training time: 0:03:53.845059
Test score: 1.4894335269927979
Test accuracy: 0.7181649804115295


Freeze only the feature layers

In [ ]:

for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 18s 71ms/step - loss: 1.5981 - accuracy: 0.2673 - val_loss: 1.5775 - val_accuracy: 0.3942
Epoch 2/5
240/240 [==============================] - 19s 77ms/step - loss: 1.5661 - accuracy: 0.3358 - val_loss: 1.5421 - val_accuracy: 0.4666
Epoch 3/5
240/240 [==============================] - 18s 74ms/step - loss: 1.5329 - accuracy: 0.3931 - val_loss: 1.5071 - val_accuracy: 0.4913
Epoch 4/5
240/240 [==============================] - 17s 72ms/step - loss: 1.5024 - accuracy: 0.4369 - val_loss: 1.4732 - val_accuracy: 0.5077
Epoch 5/5
240/240 [==============================] - 16s 68ms/step - loss: 1.4719 - accuracy: 0.4742 - val_loss: 1.4392 - val_accuracy: 0.5340
Training time: 0:01:28.558518
Test score: 1.4392324686050415
Test accuracy: 0.5339560508728027


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [ ]:
# Learned features unfrozen to use on the digits 0-4
for l in feature_layers:
    l.trainable = True

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 48s 194ms/step - loss: 1.4368 - accuracy: 0.5137 - val_loss: 1.3953 - val_accuracy: 0.5781
Epoch 2/5
240/240 [==============================] - 51s 211ms/step - loss: 1.3929 - accuracy: 0.5580 - val_loss: 1.3443 - val_accuracy: 0.6684
Epoch 3/5
240/240 [==============================] - 48s 200ms/step - loss: 1.3439 - accuracy: 0.6031 - val_loss: 1.2851 - val_accuracy: 0.7354
Epoch 4/5
240/240 [==============================] - 48s 200ms/step - loss: 1.2841 - accuracy: 0.6577 - val_loss: 1.2172 - val_accuracy: 0.7861
Epoch 5/5
240/240 [==============================] - 49s 204ms/step - loss: 1.2207 - accuracy: 0.6958 - val_loss: 1.1414 - val_accuracy: 0.8200
Training time: 0:04:04.521731
Test score: 1.141371726989746
Test accuracy: 0.820003867149353


In [ ]:
# Freezed feature layers for digits 5-9
for l in feature_layers:
    l.trainable = False

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 18s 74ms/step - loss: 1.4396 - accuracy: 0.4437 - val_loss: 1.3848 - val_accuracy: 0.5682
Epoch 2/5
230/230 [==============================] - 17s 74ms/step - loss: 1.3975 - accuracy: 0.4892 - val_loss: 1.3429 - val_accuracy: 0.6229
Epoch 3/5
230/230 [==============================] - 17s 76ms/step - loss: 1.3599 - accuracy: 0.5300 - val_loss: 1.3034 - val_accuracy: 0.6729
Epoch 4/5
230/230 [==============================] - 17s 76ms/step - loss: 1.3246 - accuracy: 0.5728 - val_loss: 1.2658 - val_accuracy: 0.7136
Epoch 5/5
230/230 [==============================] - 17s 75ms/step - loss: 1.2894 - accuracy: 0.6079 - val_loss: 1.2293 - val_accuracy: 0.7431
Training time: 0:02:22.766825
Test score: 1.2293418645858765
Test accuracy: 0.7430570125579834


#### Conclusion

#In this laboratory activity, we learned about Transfer Learning. Transfer Learning is a ML technique where a model trained on one task is reused on another task. In the activity, the model was first trained on the images of digits 0 to 4 and froze the layers to keep its learnings and then reused on the images of digits 5 to 9. We are then tasked to essentially reverse the process and found out that it is a bit more accurate compared to before. All in all, Transfer Learning is a useful technique in ML as it also helps with model performance and has faster training times.